## Algorithm Code

In [1]:
import numpy as np
import scipy.special
from math import log, floor

In [161]:
# Auxiliary functions
#################################################################################################
def shift_polynomial(coeffs, shift):
    """
    Return coefficients of a shifted polynomial
    P(x) = p(x+shift)
    Args:
    coeffs: list of float
    coefficients of p
    shift: float
    right shift of polynomial
    Returns:
    coefficients of P
    """
    length = len(coeffs)
    coeffs_shift = np.zeros(length)
    for i in range(length):
        for j in range(i+1):
            coeffs_shift[j] += coeffs[i] * (shift**(i-j)) * scipy.special.binom(i, j)
    return coeffs_shift


def hist_to_fin(hist):
    """
    Return a fingerprint from histogram
    """
    fin = {}
    for freq in hist:
        if freq in fin:
            fin[freq] += 1
        else:
            fin[freq] = 1
    return fin.items()

def samples_to_histogram(samples, n):
    """
    Return a histogram from a list of smaples, n = universe size
    """
    histogram = [0]*n
    for s in samples:
        histogram[s] += 1
    return histogram

##################################################################################################


def poly_g(L, interval, num):
    
    """
    Returns coefficient of scaled and shifted chebyshev polynomial according to eq (21) in Wu Yang
    
    Args:
    L: degree of polynomial
    interval: interval to fit polynomial to. [l,r] in Wu Yang paper
    num: number of smaples
    
    """
    
    # get interval and chebyshev coeffs
    left_end, right_end = interval
    cheb = np.polynomial.chebyshev.Chebyshev.basis(L)
    cheb_coeffs = np.polynomial.chebyshev.cheb2poly(cheb.coef)
    
    # calculate shift and scaling
    shift = (right_end + left_end) / (right_end - left_end)
    scaling = 2. / (num*right_end-num*left_end)
    

    # shift polynomial
    a_coeffs = shift_polynomial(cheb_coeffs, -shift)
    g_coeffs = -a_coeffs / a_coeffs[0]
    g_coeffs[0] = 0
    
    # scale polynomial
    for j in range(1, L+1):
        for i in range(1, j+1):
            g_coeffs[j] *= (i*scaling)
        g_coeffs[j] += 1
    
    return g_coeffs
    


def estimator(L, hist, interval):
    """
    Returns estimate of support given args
    
    Args:
    L: degree of polynomial
    hist: histogram of samples
    interval: interval we want to approximate support in

    """
    
    # converge histogram to fingerprint
    fin = hist_to_fin(hist)
    num = sum(hist)
    if (num == 0):
        return 0
    
    # get coefficient of polynomial used for estimation
    g_coeffs = poly_g(L, interval, num)
    
    # get estimate by looping over fingerprint
    supp_estimate = 0.0
    for freq, cnt in fin:
        if freq > L: # plug-in
            supp_estimate += 1*cnt
        else: # polynomial
            supp_estimate += g_coeffs[freq]*cnt
    return supp_estimate

def WuYangEstimator(n, hist, L = None, left_end = None, right_end = None):
    """
    Returns estimate of support according to Wu Yang paper specifications
    
    Args:
    n: estimate of universe size
    hist: histogram of samples
    
    Optional Args: if not given set to Wu Yang paper specification
    L: degree of polynomial
    left_end: left end of interval we want to esimate support in

    """
    
    # set Wu Yang parameters
    num = sum(hist)
    if L == None:
        L = floor(0.45*log(n))
    if left_end == None:
        left_end = 1./n
    if right_end == None:
        right_end = 0.5*log(n)/num
    
    # call estimator
    return estimator(L, hist, [left_end, right_end])


def OracleEstimator(histograms, intervals, L):
    """
    Returns sum of supports for each interval
    
    Args:
    histograms: list of histograms, one for each interval
    intervals: list of intervals where each interval is of the form [left_end, right_end]
    L: degree of polynomial

    """
    supp_estimate = 0.0
    
    # loop over histogram and intervals and add up support 
    for i, hist in enumerate(histograms):
        curr_interval = intervals[i]
        supp_estimate += estimator(L, hist, curr_interval)
    return supp_estimate
    

In [3]:
fin = [[1,2910],[2,45]]
num = 1*2910 + 2*45
pmin = 1e-5
L = floor(0.45*log(1./pmin))
left_end = pmin
right_end = 0.5*log(1./pmin)/num

In [4]:
hist = [1 for k in range(2910)] 
for i in range(45):
    hist.append(2)    


In [5]:
WuYangEstimator(1e5, hist)

23722.435544392072

## AOL Data

In [6]:
actual_data = np.load('data/query_counts_day_0050.npz') 
predicted_data = np.load('data/aol_inf_all_v05_t50_exp22_aol_5d_r1-h1_u256-32_eb64_bs128_ra_20180514-160509_ep190_res.npz')

In [8]:
print(actual_data.files)
print(predicted_data.files)
print(actual_data['queries'].shape)
print(predicted_data['test_output'].shape)

['queries', 'counts']
['test_output', 'test_loss']
(140725,)
(140725, 1)


In [9]:
actual_prob = actual_data['counts']/actual_data['counts'].sum()

In [10]:
predicted_prob = predicted_data['test_output']/predicted_data['test_output'].sum()

In [11]:
predicted_prob = predicted_prob[:,0]

In [13]:
good_indices = np.where(predicted_data['test_output'][:,0] > 0)

In [14]:
((np.log(predicted_data['test_output'][:,0][good_indices])-np.log(actual_data['counts'][good_indices]))**2).sum()

52561.20915692093

In [163]:
# Testing with AOL data

n = 1e6
# samples
S = np.random.choice(range(140725), size=50000, p=actual_prob)
# get probabilities
S_predicted_prob = actual_prob[S]
# round probs to nearest multiple of 1/n
rounded_prob = [int(round(n*k)) for k in S_predicted_prob]
S_with_prob = list(zip(S, rounded_prob))

# get intervals
eps = 1e-6
intervals = []
left = 1
right = 4
while(right < 1/eps):
    intervals.append([left, right])
    left = right
    right *= 4
    
intervals.append([left, right])


# partition samples into intervals
sample_partition = [[] for k in range(len(intervals))]
for k in S_with_prob:
    curr_prob = k[1]
    bucket = int(np.log(curr_prob)/np.log(2))-1
    try:
        sample_partition[bucket].append(k[0])
    except:
        pass
    
# get histogram for each interval
histograms = [samples_to_histogram(samp, int(n)) for samp in sample_partition]

In [164]:
print(OracleEstimator(histograms, intervals, 50))
full_hist = samples_to_histogram(S,int(n))
print(WuYangEstimator(n, full_hist))

34712.24963646076
165899.5364015394


In [159]:
# Testing with fake data
n = 10000
# samples
S = np.random.choice(range(100), size=300)
# get probabilities
S_predicted_prob = [1./100 for s in S ]
# round probs to nearest multiple of 1/n
rounded_prob = [int(round(n*k)) for k in S_predicted_prob]
S_with_prob = list(zip(S, rounded_prob))

# get intervals
eps = 1e-3
intervals = []
left = 1
right = 4
while(right < 1/eps):
    intervals.append([left, right])
    left = right
    right *= 4
    
intervals.append([left, right])

# partition samples into intervals
sample_partition = [[] for k in range(len(intervals))]
for k in S_with_prob:
    curr_prob = k[1]
    bucket = int(np.log(curr_prob)/np.log(4))-1
    try:
        sample_partition[bucket].append(k[0])
    except:
        pass
    
    
# get histogram for each interval
histograms = [samples_to_histogram(samp, n) for samp in sample_partition]

In [162]:
print(OracleEstimator(histograms, intervals, 20))
full_hist = samples_to_histogram(S,n)
print(WuYangEstimator(n, full_hist))

91.01865656855415
19.01295015441407
